In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision 
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor
import time
from collections import OrderedDict

In [2]:
# Preparing for Data
print('==> Preparing data..')
#classes = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')

==> Preparing data..


In [3]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()

        self.convnet = nn.Sequential(OrderedDict([
            ('c1',nn.Conv2d(1,6,kernel_size=(5,5))),
            ('relu1',nn.ReLU()),
            ('s2',nn.MaxPool2d(kernel_size=(2,2), stride=2)),
            ('c3',nn.Conv2d(6,16,kernel_size=(5,5))),
            ('relu3',nn.ReLU()),
#             ('drop3',nn.Dropout2d(p=0.05)),
            ('s4',nn.MaxPool2d(kernel_size=(2,2), stride=2))
        ]))
        
        self.fc = nn.Sequential(OrderedDict([
            ('f5',nn.Linear(256,120)),
            ('relu5',nn.ReLU()),
            ('f6',nn.Linear(120,84)),
            ('relu6',nn.ReLU()),
            ('f7',nn.Linear(84,10)),
            ('sig7',nn.LogSoftmax(dim=-1))
        ]))

    def forward(self, x):

        # print(len(x[0]))
        out = self.convnet(x)
        # print(out.size(),len(out[0]))
        # x=x.view(128,120)
        out = out.view(out.size(0), -1)
        out = self.fc(out)

        return out


In [4]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    count = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output  = model(data)
        # loss = F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
        criterion = nn.CrossEntropyLoss() # sum up batch loss
        loss = criterion(output,target)
        loss.backward()
        optimizer.step()

        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test( model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [5]:
def main():
    time0 = time.time()
    # Training settings
    batch_size = 128
    epochs = 50
    lr = 0.05
    no_cuda = True
    save_model = False
    use_cuda = not no_cuda and torch.cuda.is_available()
    torch.manual_seed(100)
    device = torch.device("cuda" if use_cuda else "cpu")
    
    trainset = torchvision.datasets.MNIST(root='./train', train=True, download=True, transform=ToTensor())
    train_loader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)
    testset = torchvision.datasets.MNIST(root='./test', train=False, download=True, transform=ToTensor())
    test_loader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False)

    model = LeNet().to(device)
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)

    for epoch in range(1, epochs + 1):
        train( model, device, train_loader, optimizer, epoch)
        test( model, device, test_loader)

    if (save_model):
        torch.save(model.state_dict(),"cifar_lenet.pt")
    time1 = time.time() 
    print ('Traning and Testing total excution time is: %s seconds ' % (time1-time0))   
if __name__ == '__main__':
    main()

Extracting ./train/MNIST/raw/train-images-idx3-ubyte.gz to ./train/MNIST/raw


Extracting ./train/MNIST/raw/train-labels-idx1-ubyte.gz to ./train/MNIST/raw


Extracting ./train/MNIST/raw/t10k-images-idx3-ubyte.gz to ./train/MNIST/raw


Extracting ./train/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./train/MNIST/raw
Processing...
Done!




Extracting ./test/MNIST/raw/train-images-idx3-ubyte.gz to ./test/MNIST/raw


Extracting ./test/MNIST/raw/train-labels-idx1-ubyte.gz to ./test/MNIST/raw


Extracting ./test/MNIST/raw/t10k-images-idx3-ubyte.gz to ./test/MNIST/raw


Extracting ./test/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./test/MNIST/raw
Processing...
Done!



Train Epoch: 1 [0/60000 (0%)]	Loss: 2.305031
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.298522
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.284686
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.261117
Train Epoch: 1 [5120/60000 (9%)]	Loss: 1.963376
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.655745
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.951834
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.599735
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.770311
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.340462
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.298734
Train Epoch: 1 [14080/60000 (23%)]	Loss: 0.291193
Train Epoch: 1 [15360/60000 (26%)]	Loss: 0.268067
Train Epoch: 1 [16640/60000 (28%)]	Loss: 0.288113
Train Epoch: 1 [17920/60000 (30%)]	Loss: 0.259382
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.216780
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.151267
Train Epoch: 1 [21760/60000 (36%)]	Loss: 0.138235
Train Epoch: 1 [230

KeyboardInterrupt: 